In [3]:
train_path='/kaggle/input/legal-train-1/train_part_1.csv'
corpus_path='/kaggle/input/hackathon-2024-legal/corpus.csv'
tokenize_corpus='/kaggle/input/corpus-tokenization/tokenized_corpus.pkl'

In [4]:
!pip install rank_bm25 underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 64.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00


In [5]:
from rank_bm25 import BM25Okapi
import pandas as pd
import pickle
from underthesea import word_tokenize

In [6]:
corpus=pd.read_csv(corpus_path)
corpus.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê\n...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê\n...",4


In [7]:
corpus_text=corpus['text'].to_list()
corpus_cid=corpus['cid'].to_list()
print(corpus_text[0])
print(corpus_cid[0])
# corpus_tokenization=[doc.split(" ") for doc in corpus_text]
# corpus_tokenization=[word_tokenize(doc) for doc in corpus_text]
# print(corpus_tokenization[0])

Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.
0


In [9]:
with open(tokenize_corpus, 'rb') as f:
    corpus_tokenization = pickle.load(f)

In [10]:
legal_bm25 = BM25Okapi(corpus_tokenization)

In [11]:
train_data=pd.read_csv(train_path)
data_records=train_data.to_dict(orient='records')

for point in data_records:
    cid_str=point['cid']
    point['cid'] = list(map(int, cid_str.strip('[]').split()))
print(data_records[0])

{'question': 'Người học ngành quản lý khai thác công trình thủy lợi trình độ cao đẳng phải có khả năng học tập và nâng cao trình độ như thế nào?', 'context': "['Khả năng học tập, nâng cao trình độ\\n- Khối lượng kiến thức tối thiểu, yêu cầu về năng lực mà người học phải đạt được sau khi tốt nghiệp ngành, nghề quản lý, khai thác các công trình thủy lợi, trình độ cao đẳng có thể tiếp tục phát triển ở các trình độ cao hơn;\\n- Người học sau tốt nghiệp có năng lực tự học, tự cập nhật những tiến bộ khoa học công nghệ trong phạm vi ngành, nghề để nâng cao trình độ hoặc học liên thông lên trình độ cao hơn trong cùng ngành, nghề hoặc trong nhóm ngành, nghề hoặc trong cùng lĩnh vực đào tạo./.']", 'cid': [62492], 'qid': 161615}


In [12]:
# Khởi tạo danh sách tạm để lưu trữ kết quả
temp = []
# Số lượng điểm tối đa trước khi lưu vào file
batch_size = 10

for idx, point in enumerate(data_records):
    top_relevant = legal_bm25.get_top_n(word_tokenize(point['question']), corpus_cid, 10)
    hard_negatives = [item for item in top_relevant if item not in point['cid']]  
    curr=[point['qid']]
    curr.extend(hard_negatives)
    # Thêm question_id và hard_negatives vào danh sách tạm
    temp.append(curr)

    # Kiểm tra nếu đã đạt đến số lượng batch_size
    if (idx + 1) % batch_size == 0:
        # Ghi kết quả vào file
        with open('/kaggle/working/hard_negative.txt', 'a') as f:
            for item in temp:
                f.write(f"{item}\n")
        print(f'inserting successfully {idx+1} elements')
        temp.clear()

# Lưu kết quả còn lại sau vòng lặp nếu có
if temp:
    with open('/kaggle/working/hard_negative.txt', 'a') as f:
        for item in temp:
            f.write(f"{item}\n")



inserting successfully 9 elements
inserting successfully 19 elements


KeyboardInterrupt: 